Para los datos de entrenamiento vamos a hacer un etiquetado sintético del color de las imágenes.

In [6]:
import tensorflow as tf
from tqdm.keras import TqdmCallback
import numpy as np
import os

# --- 1. CONFIGURACIÓN ---
IMG_SIZE = (96, 96)
BATCH_SIZE = 32
SEED = 42

train_path = "/kaggle/input/stl10/unlabeled_images"
test_path = "/kaggle/input/stl10/test_images"

# --- 2. LÓGICA DE PROMPTS SINTÉTICOS ---
# Paleta para detectar el color dominante
COLOR_PALETTE = {
    "red":   np.array([1.0, 0.0, 0.0]),
    "green": np.array([0.0, 1.0, 0.0]),
    "blue":  np.array([0.0, 0.0, 1.0]),
    "yellow":np.array([1.0, 1.0, 0.0]),
    "cyan":  np.array([0.0, 1.0, 1.0]),
    "magenta":np.array([1.0, 0.0, 1.0]),
    "white": np.array([1.0, 1.0, 1.0]),
    "black": np.array([0.0, 0.0, 0.0]),
    "gray":  np.array([0.5, 0.5, 0.5]),
    "orange":np.array([1.0, 0.5, 0.0]),
    "purple":np.array([0.5, 0.0, 0.5]),
    "brown": np.array([0.6, 0.4, 0.2])
}
palette_keys = list(COLOR_PALETTE.keys())
palette_vals = np.array(list(COLOR_PALETTE.values()), dtype=np.float32)

def generate_prompt(img_rgb):
    """Calcula color medio y devuelve string (ej: 'color red')"""
    mean_color = np.mean(img_rgb, axis=(0, 1)) 
    distances = np.linalg.norm(palette_vals - mean_color, axis=1)
    closest_index = np.argmin(distances)
    color_name = palette_keys[closest_index]
    return "color " + color_name

# --- 3. PIPELINE DE DATOS ---
def process_image(file_path):
    # Cargar y decodificar
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    
    # Input 1: Escala de grises
    gray = tf.image.rgb_to_grayscale(img)
    
    # Input 2: Prompt de texto (generado al vuelo)
    prompt = tf.py_function(func=generate_prompt, inp=[img], Tout=tf.string)
    prompt = tf.expand_dims(prompt, axis=0) 
    prompt.set_shape((1,)) # Necesario para Keras
    
    # Estructura: ((Inputs), Target)
    return (gray, prompt), img

In [7]:
# Crear Datasets
train_files = tf.data.Dataset.list_files(os.path.join(train_path, "*.png"), seed=SEED)
# Nota: Usamos take() si el dataset es gigante para probar rápido, quítalo para entrenar completo
train_ds = train_files.take(20000).map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_files = tf.data.Dataset.list_files(os.path.join(test_path, "*.png"), seed=SEED)
test_ds = test_files.take(1000).map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [11]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input

def residual_block(x, filters, kernel_size=3):
    res = x
    x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same', activation=None)(x)
    x = layers.BatchNormalization()(x)
    if res.shape[-1] != filters:
        res = layers.Conv2D(filters, 1, padding='same')(res)

    x = layers.Add()([x, res])
    x = layers.Activation('relu')(x)
    return x

def text_guided_AE(input_shape=(96,96,1)):
    
    # --- ENTRADAS ---
    img_input = Input(shape=input_shape, name='image_input')
    text_input = Input(shape=(1,), dtype=tf.string, name='text_input')

    # --- RAMA DE TEXTO ---
    # Convertimos texto a vector numérico
    # Nota: max_tokens=20 porque solo usaremos colores simples ("red", "dark blue", etc)
    vectorizer = layers.TextVectorization(max_tokens=20, output_mode='int', output_sequence_length=1)
    
    # Embedding: Convierte el índice del token en un vector rico de 128 valores
    t = vectorizer(text_input)
    t = layers.Embedding(input_dim=20, output_dim=128)(t)
    t = layers.Lambda(lambda x: tf.squeeze(x, axis=1), name='squeeze_text_embedding')(t) # Vector de forma (Batch, 128)
    
    # Preparamos el texto para inyectarlo en la imagen
    # Queremos que el texto afecte a toda la imagen espacialmente.
    # En el bottleneck, la imagen mide 12x12. Repetimos el texto para que coincida.
    t_repeated = layers.RepeatVector(12 * 12)(t) 
    t_spatial = layers.Reshape((12, 12, 128))(t_repeated) # Forma (Batch, 12, 12, 128)

    # --- RAMA DE IMAGEN (ENCODER) ---
    # Pre-procesamiento inicial
    x = layers.Conv2D(64, (3,3), padding='same', activation='relu')(img_input)
    
    # Bloque 1
    x1 = residual_block(x, 64)
    p1 = layers.MaxPooling2D((2,2))(x1) # Sale 48x48

    # Bloque 2
    x2 = residual_block(p1, 128)
    p2 = layers.MaxPooling2D((2,2))(x2) # Sale 24x24

    # Bloque 3
    x3 = residual_block(p2, 256)
    p3 = layers.MaxPooling2D((2,2))(x3) # Sale 12x12

    # --- FUSIÓN (BOTTLENECK) ---
    # Aquí unimos la imagen comprimida (p3) con el texto espacial (t_spatial)
    # p3 shape: (12, 12, 256)
    # t_spatial shape: (12, 12, 128)
    combined = layers.Concatenate()([p3, t_spatial]) # Shape resultante: (12, 12, 384)
    
    # Pasamos la fusión por el bloque residual del bottleneck
    # Esto permite que la red aprenda cómo el texto debe alterar las características visuales
    b = residual_block(combined, 512)

    # --- DECODER (Simétrico) ---
    
    # Subida 3 (De 12x12 a 24x24)
    u3 = layers.Conv2DTranspose(256, (3,3), strides=(2,2), padding='same', activation='relu')(b)
    # Concatenamos con x3 (la salida del encoder antes del pooling)
    c3 = layers.Concatenate()([u3, x3]) 
    d3 = residual_block(c3, 256)

    # Subida 2 (De 24x24 a 48x48)
    u2 = layers.Conv2DTranspose(128, (3,3), strides=(2,2), padding='same', activation='relu')(d3)
    c2 = layers.Concatenate()([u2, x2])
    d2 = residual_block(c2, 128)

    # Subida 1 (De 48x48 a 96x96)
    u1 = layers.Conv2DTranspose(64, (3,3), strides=(2,2), padding='same', activation='relu')(d2)
    c1 = layers.Concatenate()([u1, x1])
    d1 = residual_block(c1, 64)

    # --- OUTPUT ---
    # Salida RGB (3 canales), Sigmoid para rango [0, 1]
    outputs = layers.Conv2D(3, (1,1), activation='sigmoid', dtype='float32', name='rgb_output')(d1)

    model = models.Model(inputs=[img_input, text_input], outputs=outputs)
    return model, vectorizer

# Instanciar 
model, vectorizer = text_guided_AE()


In [12]:
print("Adaptando vocabulario de texto...")

text_ds_for_adapt = train_ds.unbatch().map(lambda inputs, target: tf.reshape(inputs[1], [-1])).take(1000)
# En el map: inputs[1] es (B, 1) -> tf.reshape(..., [-1]) lo convierte a (B,)

vectorizer.adapt(text_ds_for_adapt)
# ---------------------------------------------

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

print("Vocabulario adaptado con éxito. El modelo está listo para el entrenamiento.")

Adaptando vocabulario de texto...
Vocabulario adaptado con éxito. El modelo está listo para el entrenamiento.


In [15]:
print("Empezando entrenamiento...")
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=100,
    verbose=0,
    callbacks=[TqdmCallback(verbose=1)]
)

print("Entrenamiento finalizado.")

Empezando entrenamiento...


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

Entrenamiento finalizado.


In [16]:
# --- CÓDIGO PARA GUARDAR EL MODELO ---

model.save('text_guided_colorizer.keras') 
print("Modelo guardado con éxito como 'text_guided_colorizer.keras'")

Modelo guardado con éxito como 'text_guided_colorizer.keras'


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

# --- 1. CONFIGURACIÓN DE PRUEBA ---
# Importamos el modelo (si lo guardaste y lo cargas en otra sesión)
# from tensorflow.keras.models import load_model
# model = load_model('text_guided_colorizer.keras')

# Definimos el prompt de color a probar (¡Esta es la guía del usuario!)
TEST_PROMPT = "color blue" 
TEST_BATCH_SIZE = 32 # Debe coincidir con el BATCH_SIZE usado en el dataset

# --- 2. PREPARAR DATOS DE ENTRADA ---

# Tomar un batch del test_dataset (asumiendo que 'test_ds' está definido y cargado)
# El test_ds debe tener la estructura: ((gray, text_prompt), color_rgb)
for (gray_batch, text_batch), true_color_batch in test_ds.take(1):
    gray_input = gray_batch        # (BATCH, 96, 96, 1)
    true_color = true_color_batch  # (BATCH, 96, 96, 3)
    break

# Creamos un tensor de prompts repetidos para todo el lote de prueba
# Usamos el prompt de prueba definido por el usuario (TEST_PROMPT)
user_prompts = tf.constant([TEST_PROMPT] * TEST_BATCH_SIZE) # Forma (BATCH_SIZE,)

# Necesitamos expandir la dimensión para que coincida con el Input del modelo (B, 1)
# Si en tu adaptador tenías que el input era (B, 1), esto funciona:
text_input_for_model = tf.expand_dims(user_prompts, axis=1) 
# Si tu vectorizer ya maneja (B,), puedes usar user_prompts directamente, 
# pero por seguridad, forzamos la forma (B, 1) para el modelo.

# --- 3. HACER PREDICCIÓN CON DOBLE INPUT ---
# El modelo espera [Imagen, Texto]
predicted_colors = model.predict([gray_input, text_input_for_model])

# --- 4. FUNCIÓN DE PLOTEO ---
def show_result_guided(i, predictions, gray_data, real_data, prompt):
    plt.figure(figsize=(12, 4))
    
    # Imagen de entrada (grayscale)
    plt.subplot(1, 3, 1)
    # Usamos .numpy().squeeze() para eliminar dimensiones extra si existen
    plt.imshow(gray_data[i].numpy().squeeze(), cmap="gray") 
    plt.title(f"Input (Gray) | Prompt: {prompt}")
    plt.axis("off")

    # Predicción del modelo
    plt.subplot(1, 3, 2)
    # Las predicciones ya están en [0, 1] RGB
    plt.imshow(predictions[i])
    plt.title("Predicted Color")
    plt.axis("off")

    # Imagen real (Ground Truth)
    plt.subplot(1, 3, 3)
    plt.imshow(real_data[i])
    plt.title("Real Color")
    plt.axis("off")

    plt.suptitle(f"Prueba con Guía de Texto: '{prompt}'", fontsize=14)
    plt.show()

# --- 5. MOSTRAR RESULTADOS ---

# Muestra 5 ejemplos del lote, todos coloreados con el mismo prompt
for i in range(5):
    show_result_guided(i, predicted_colors, gray_input, true_color, TEST_PROMPT)

print(f"Predicciones mostradas usando el prompt guía: '{TEST_PROMPT}'")